# Callbacks

In [2]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext tensorboard

## Hyperparameter Tunning

In [4]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [5]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [6]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [6]:
os.listdir('dataset/cifar/')

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'dataset/cifar/'

In [7]:
train_paths = glob(r'C:\Users\LG\Deep_Learning_Study\Tensorflow\dataset\cifar\train\*.png')[:1000]  # 보기용으로 데이터 갯수를 1000개로 제한
test_paths = glob(r'C:\Users\LG\Deep_Learning_Study\Tensorflow\dataset\cifar\test\*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [8]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [9]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [10]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [11]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [12]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [15]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Callbacks

In [16]:
starttime = datetime.now().strftime('%Y%m%d-%H%M%S')
starttime

'20200121-185319'

In [17]:
endtime = datetime.now().strftime('%Y%m%d-%H%M%S')
endtime

'20200121-185319'

In [18]:
endtime - starttime

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [19]:
logdir = os.path.join('logs', datetime.now().strftime('%Y%m%d-%H%M%S'))

In [20]:
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    write_graph=True,
    write_images=True,
    histogram_freq=1
)

https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/image_summaries.ipynb#scrollTo=IJNpyVyxbVtT

In [27]:
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4280.

In [30]:
%tensorboard --logdir logs --port 8080

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 15548.

## Training

http://localhost:6006

In [21]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard]
)

Epoch 1/10
31/31 [==============================] - 12s 390ms/step - loss: 2.3126 - accuracy: 0.1095 - val_loss: 2.2895 - val_accuracy: 0.1502
Epoch 2/10
31/31 [==============================] - 12s 372ms/step - loss: 2.2370 - accuracy: 0.1601 - val_loss: 2.1585 - val_accuracy: 0.2248
Epoch 3/10
31/31 [==============================] - 11s 355ms/step - loss: 2.1585 - accuracy: 0.1996 - val_loss: 2.1263 - val_accuracy: 0.1925
Epoch 4/10
31/31 [==============================] - 10s 338ms/step - loss: 2.1428 - accuracy: 0.2225 - val_loss: 2.0981 - val_accuracy: 0.1925
Epoch 5/10
31/31 [==============================] - 11s 349ms/step - loss: 2.0499 - accuracy: 0.2200 - val_loss: 2.0247 - val_accuracy: 0.2369
Epoch 6/10
31/31 [==============================] - 10s 334ms/step - loss: 2.0393 - accuracy: 0.2479 - val_loss: 2.1096 - val_accuracy: 0.2228
Epoch 7/10
31/31 [==============================] - 11s 349ms/step - loss: 2.0611 - accuracy: 0.2298 - val_loss: 1.9930 - val_accuracy: 0.2581

https://www.tensorflow.org/tensorboard/r2/image_summaries#setup

### LambdaCallback

In [31]:
import sklearn.metrics
import itertools
import io

file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Normalize the confusion matrix.
  cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [32]:
test_images, test_labels = next(iter(test_dataset))

def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)


In [33]:
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [34]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[tensorboard, cm_callback]
)

Epoch 1/10
30/31 [============================>.] - ETA: 0s - loss: 1.7555 - accuracy: 0.3526

C:\Users\LG\Anaconda3\envs\py3.6ten2.0\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 11s 370ms/step - loss: 1.7520 - accuracy: 0.3585 - val_loss: 1.8426 - val_accuracy: 0.3135
Epoch 2/10
30/31 [============================>.] - ETA: 0s - loss: 1.7224 - accuracy: 0.3707

C:\Users\LG\Anaconda3\envs\py3.6ten2.0\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 12s 391ms/step - loss: 1.7323 - accuracy: 0.3657 - val_loss: 1.7962 - val_accuracy: 0.3357
Epoch 3/10
30/31 [============================>.] - ETA: 0s - loss: 1.7386 - accuracy: 0.3526

...

31/31 [==============================] - 12s 391ms/step - loss: 1.6031 - accuracy: 0.4225 - val_loss: 1.7200 - val_accuracy: 0.3871
Epoch 9/10
30/31 [============================>.] - ETA: 0s - loss: 1.6280 - accuracy: 0.3921

C:\Users\LG\Anaconda3\envs\py3.6ten2.0\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 12s 382ms/step - loss: 1.6320 - accuracy: 0.3936 - val_loss: 1.6792 - val_accuracy: 0.3750
Epoch 10/10
30/31 [============================>.] - ETA: 0s - loss: 1.5616 - accuracy: 0.4333

C:\Users\LG\Anaconda3\envs\py3.6ten2.0\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


31/31 [==============================] - 12s 373ms/step - loss: 1.5618 - accuracy: 0.4294 - val_loss: 1.6953 - val_accuracy: 0.3972


## Expert

In [35]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [36]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [37]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)  # Loss 계산
    gradients = tape.gradient(loss, model.trainable_variables)  # 모델의 trainable_variable을 하여금 loss를 통해 기울기를 얻음
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # 구한 최적화된 값을 variable에 적용

    train_loss(loss)
    train_accuracy(labels, predictions)

In [38]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [39]:
logdir = os.path.join('logs',  datetime.now().strftime("%Y%m%d-%H%M%S"))

In [40]:
file_writer = tf.summary.create_file_writer(logdir)

In [41]:
for epoch in range(num_epochs):
    for step, (images, labels) in enumerate(train_dataset):            
        train_step(images, labels)

    with file_writer.as_default():
        tf.summary.image('input_image', images, step = step + (step * epoch))
        tf.summary.scalar('train_loss', train_loss.resuit(), step = step + (step* epoch))
        tf.summary.scalar('train_accuracy', train_accuracy.resuit(), step = step + (step* epoch))
        
    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))

KeyboardInterrupt: 